In [1]:
%pylab inline
import sys; sys.path.insert(0, "../")
import numpy as np
import timeit
from emcee.autocorr import *
from ensemble_sampler import *

Populating the interactive namespace from numpy and matplotlib


### Example of sampling from an invariant distribution of SPDE  
Ref. Goodman & Weare, Ensemble Samplers With Affine Invariance  
A problem with this example is that it's harder to visualize, but from the experiments it does seem that pCN has more consistent performance in high dimensions (judging from acceptance probability).

In [2]:
# Fixed for all experiments in this notebook
nwalkers = 100
batch_size = 50

In [3]:
def plot_traj(x):
    n, dim = x.shape
    for i in range(n):
        t = np.linspace(start=0.0, stop=1.0, num=dim)
        plt.plot(t, x[i, :])

In [4]:
def run(dim, proposal, batch_size=50, niters=1000, n=1, pre=0, verbose=False, print_every=1):
    t_dist = SPDE(N=dim)
    sampler = Sampler(dim=dim, t_dist=t_dist, proposal=proposal, nwalkers=nwalkers)

    acc_r = 0.0
    for i in range(n):
        sampler.reset()
        p0 = np.random.randn(dim*nwalkers).reshape([nwalkers, dim])
        if pre > 0:
            hist = sampler.run_mcmc(pre, batch_size=batch_size, p0=p0, verbose=False)
            p0 = hist.curr_pos
            sampler.reset()
        start = timeit.default_timer()
        hist = sampler.run_mcmc(niters, batch_size=batch_size, p0=p0, verbose=verbose, print_every=print_every)
        print 'finishes loop %d in %.2f seconds' % (i, float(timeit.default_timer() - start))
        acc_r += float(100*hist.acceptance_rate.mean())
        print 'avg accept rate: %.2f%s' % (float(100*hist.acceptance_rate.mean()), '%')
        try:
            print 'mean auto-correlation time: %s' % sampler.auto_corr()
        except AutocorrError, err:
            print err
    print 'avg_acc_r: %.2f%s' % (float(acc_r) / 5.0, '%')

#### Not using ensemble, use isotropic gaussian proposal with scale=0.04

In [5]:
dim = 50
proposal = PCNWalkMove(s=None, scale=0.04)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 28.44 seconds
avg accept rate: 31.06%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 28.97 seconds
avg accept rate: 30.98%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 28.30 seconds
avg accept rate: 31.10%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 28.95 seconds
avg accept rate: 29.76%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 28.77 seconds
avg accept rate: 29.64%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 30.51%


#### Use ensemble of size 3, scale=0.008.  
Experimented with dim=50 & 100, see a decrease in acceptance rate with same scaling parameter.

In [15]:
dim = 50
proposal = PCNWalkMove(s=3, scale=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 31.37 seconds
avg accept rate: 53.29%
mean auto-correlation time: 564.171299344
finishes loop 1 in 33.02 seconds
avg accept rate: 53.02%
mean auto-correlation time: 437.66296795
finishes loop 2 in 31.48 seconds
avg accept rate: 53.66%
mean auto-correlation time: 534.626637265
finishes loop 3 in 29.19 seconds
avg accept rate: 52.89%
mean auto-correlation time: 76.6306596799
finishes loop 4 in 33.09 seconds
avg accept rate: 53.98%
mean auto-correlation time: 305.558843689
avg_acc_r: 53.37%


In [16]:
dim = 100
proposal = PCNWalkMove(s=3, scale=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 40.39 seconds
avg accept rate: 55.57%
mean auto-correlation time: 574.424123777
finishes loop 1 in 40.40 seconds
avg accept rate: 54.64%
mean auto-correlation time: 663.720794119
finishes loop 2 in 36.10 seconds
avg accept rate: 56.11%
mean auto-correlation time: 539.627060716
finishes loop 3 in 37.34 seconds
avg accept rate: 55.67%
mean auto-correlation time: 643.975897747
finishes loop 4 in 42.27 seconds
avg accept rate: 56.61%
mean auto-correlation time: 527.188603762
avg_acc_r: 55.72%


In [17]:
dim = 200
proposal = PCNWalkMove(s=3, scale=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 62.94 seconds
avg accept rate: 33.19%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 57.27 seconds
avg accept rate: 33.36%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 55.49 seconds
avg accept rate: 33.29%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 55.75 seconds
avg accept rate: 32.87%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 53.13 seconds
avg accept rate: 33.28%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 33.20%


#### Ensemble with pCN, beta=0.8  
Experimented with dim=50 & 100, consistency in acceptance rate with same scaling parameter.

In [9]:
dim = 50
proposal = PCNWalkMove(s=3, beta=0.8)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 31.67 seconds
avg accept rate: 79.92%
mean auto-correlation time: 4.07545758871
finishes loop 1 in 31.28 seconds
avg accept rate: 79.97%
mean auto-correlation time: 8.332973422
finishes loop 2 in 33.57 seconds
avg accept rate: 79.97%
mean auto-correlation time: 7.78954333734
finishes loop 3 in 31.81 seconds
avg accept rate: 79.99%
mean auto-correlation time: 6.81990726261
finishes loop 4 in 33.37 seconds
avg accept rate: 79.97%
mean auto-correlation time: 3.51835596709
avg_acc_r: 79.96%


In [10]:
dim = 100
proposal = PCNWalkMove(s=3, beta=0.8)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 40.24 seconds
avg accept rate: 79.97%
mean auto-correlation time: 8.13227731892
finishes loop 1 in 43.37 seconds
avg accept rate: 79.97%
mean auto-correlation time: 4.77334864322
finishes loop 2 in 39.50 seconds
avg accept rate: 79.97%
mean auto-correlation time: 3.62875493535
finishes loop 3 in 40.62 seconds
avg accept rate: 79.96%
mean auto-correlation time: 12.8032870811
finishes loop 4 in 38.20 seconds
avg accept rate: 79.95%
mean auto-correlation time: 5.25723287241
avg_acc_r: 79.96%


In [11]:
dim = 200
proposal = PCNWalkMove(s=3, beta=0.8)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 57.42 seconds
avg accept rate: 79.94%
mean auto-correlation time: 13.8362569007
finishes loop 1 in 56.51 seconds
avg accept rate: 79.91%
mean auto-correlation time: 2.07240886433
finishes loop 2 in 57.66 seconds
avg accept rate: 79.90%
mean auto-correlation time: 14.0755356523
finishes loop 3 in 58.91 seconds
avg accept rate: 79.91%
mean auto-correlation time: 14.3212786627
finishes loop 4 in 56.23 seconds
avg accept rate: 79.94%
mean auto-correlation time: 12.8382923609
avg_acc_r: 79.92%


#### PCN without ensemble, beta=0.001  
Need small beta to get good acceptance rate.

In [18]:
dim = 50
proposal = PCNWalkMove(s=None, beta=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 30.34 seconds
avg accept rate: 77.52%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 29.33 seconds
avg accept rate: 77.51%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 29.09 seconds
avg accept rate: 77.51%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 30.59 seconds
avg accept rate: 77.48%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 28.96 seconds
avg accept rate: 77.57%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 77.52%


In [19]:
dim = 100
proposal = PCNWalkMove(s=None, beta=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 42.79 seconds
avg accept rate: 57.34%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 43.37 seconds
avg accept rate: 57.34%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 42.08 seconds
avg accept rate: 57.37%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 42.30 seconds
avg accept rate: 57.39%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 41.25 seconds
avg accept rate: 57.37%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 57.36%


In [20]:
dim = 200
proposal = PCNWalkMove(s=None, beta=0.008)

run(dim, proposal, batch_size=batch_size, niters=50000, n=5)

finishes loop 0 in 70.79 seconds
avg accept rate: 28.27%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 70.75 seconds
avg accept rate: 28.21%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 69.77 seconds
avg accept rate: 28.24%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 73.16 seconds
avg accept rate: 28.27%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 75.61 seconds
avg accept rate: 28.28%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 28.26%
